In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, DatasetDict
import pandas as pd

/opt/miniconda3/envs/uni311/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## SETUP MODEL


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map=device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


## Initialize PROMETHEUS
Since the PROMETHEUS model used with VLLM has 7 billion parameters, we attempted to load it using 
the Hugging Face Transformers library and then quantize it in order to reduce memory usage and improve inference efficiency.   
To address this:

 - We used the Hugging Face Transformers library to load the model, as it provides a standardized interface for accessing pretrained weights and integrating them into existing pipelines.

 - We then applied quantization, a common technique that reduces the numerical precision of the model weights , with the goal of:

        - Lowering memory consumption

        - Speeding up inference

        - Maintaining reasonable accuracy



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16", 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"  
)

model_id = "prometheus-eval/prometheus-7b-v2.0"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  
    quantization_config=quant_config
)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_id)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
from prometheus import PrometheusEval_AtM

evaluator = PrometheusEval_AtM(quantized = True, device = device)

In [5]:
df = pd.read_csv("test.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

dataset = Dataset.from_pandas(df)
#dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=41)


# row = df.iloc[12] # from 10 ahead to avoid using sentences given as examples 

# author = row["Author"]
# date = row["Date"]
# region = row["Region"]
# sentence = row["Sentence"]


## NON IN-CONTEXT LEARNING

In [6]:
system_prompt = "Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo."

In [ ]:
i = 1

source_sentences = []
predicted_sentences = []
gold_sentences = [] #annoted by hand
df_col = ["source_sentences", "predicted_sentences", "gold_sentences"]


for sample in dataset:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]

   
    user_prompt = f"Modernizza il seguente testo antico in italiano contemporaneo:\n\n\"{input_sentence}\"\n\n scrivi solo ed esclusivamente la traduzione"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    output = pipe(messages, max_new_tokens=300)
    translation = output[0]["generated_text"][-1]["content"]

    evaluation = evaluator.getEvaluation(input_sentence, translation, target_sentence)

    source_sentences.append(input_sentence)
    predicted_sentences.append(translation)
    gold_sentences.append(target_sentence)


    df = pd.DataFrame()
    df[df_col] = [source_sentences, predicted_sentences, gold_sentences]
    

    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico -> {input_sentence}")
    print(f"\tItaliano moderno -> {translation}")
    print(f"\tGOLD LABEL       -> {target_sentence}")
    print(f"\tEVALUATION       -> {evaluation}")
    print(f"-----------------------------------------")
    i+=1

KeyboardInterrupt: 

## IN-CONTEXT LEARNING

In [ ]:
system_prompt = """ Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi:

Testo antico: "quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno"
Traduzione moderna: "Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio."

Testo antico: "crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi."
Traduzione moderna: "È crudele e si vendica di ogni colpa, come stabilisce la legge, e non perdona alcun cavaliere che sbagli."

Testo antico: "Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere."
Traduzione moderna: "Ponzio Aufidiano, cavaliere romano, non era dotato di maggiore coraggio."

Testo antico: "Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati."
Traduzione moderna: "Se questo è il volere di tutti, e se i tempi richiedono Pompeo come guida e non come alleato, allora non tratterrò più il destino."

Testo antico: "Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire."
Traduzione moderna: "Il compito di quest'arte sembra essere quello di parlare con astuzia per far credere; lo scopo è dunque persuadere attraverso il linguaggio."
"""

In [12]:
i = 1
for sample in dataset["test"]:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]


    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico -> {input_sentence}")
    
    user_prompt = f"""Modernizza il seguente testo antico in italiano contemporaneo.
    Contesto: Autore: {author}, Data: {date}, Regione: {region}
    Testo antico: "{input_sentence}"

    Scrivi solo ed esclusivamente la traduzione moderna:"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    output = pipe(messages, max_new_tokens=300)
    translation = output[0]["generated_text"][-1]["content"]

    print(f"\tItaliano moderno -> {translation}")
    print(f"\tGOLD LABEL       -> {target_sentence}")

    print(f"-----------------------------------------")
    i+=1

Sentence 1
	Italiano Arcaico -> l'armi et insieme con loro passaseno tra li nimici, perçò se alcuno non avesse ardire de questo et sì avevano questo animo.


KeyboardInterrupt: 